# 다변수의 time series LSTM 예측-(1)Market

In [1]:
import pandas as pd
import numpy as np
from pandas import read_csv
from datetime import datetime
from pandas import DataFrame
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot

In [ ]:
m=pd.read_csv('2019빅데이터페스티벌데이터/market_noDram_final.csv')
k=pd.read_csv('2019빅데이터페스티벌데이터/market_noDram_final.csv')